# Generate result (in .ds file) based on input

# 1. Setup environment

In [3]:
import pandas as pd
import numpy as np
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 1. Lyric Input
Make sure the lyric is been preprocessed in GPT

In [ ]:
# Load the encoding dictionary from the ph_token_to_int.json file
with open('ph_token_to_int.json', 'r') as file:
    ph_token_to_int = json.load(file)

# Your input sequence
input_sequence = "n ei f a g e n a j i f u y u a p u AP n ei f a g e n a l e y u d ao en AP"

# Split the input sequence into individual phonemes
input_phonemes = input_sequence.split()

# Convert the phonemes to their corresponding integers using the encoding dictionary
encoded_sequence = [ph_token_to_int[phoneme] for phoneme in input_phonemes]

# Convert the sequence to a numpy array and pad it to the right length
new_ph_seq_encoded = np.array([encoded_sequence])  # wrapping in a list to create a batch dimension
new_ph_seq_encoded_padded = pad_sequences(new_ph_seq_encoded, maxlen=max_sequence_length, padding='post')

# If you normalized your data, apply the same normalization to the new input
new_ph_seq_encoded_normalized = scaler_ph_seq.transform(new_ph_seq_encoded_padded)  # replace 'scaler_ph_seq' with the actual scaler used


# 2. Inference
Generate result

In [ ]:
predicted_ph_dur, predicted_f0_seq, predicted_note_seq_encoded = model.predict(new_ph_seq_encoded_normalized)